In [1]:
from scipy.sparse import csc_array
from scipy.sparse.linalg import bicgstab
from scipy.sparse.linalg import cg
from scipy.sparse.linalg import gmres
import numpy as np
import matplotlib.pyplot as plt
pi = np.pi

In [2]:
def DatosMat(path):
    data = [[],[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return data

def DatosVec(path):
    data = [[],[],[]]
    with open(path) as fobj:
        for i, line in enumerate(fobj):
           # data.append([])
            if line.rsplit() != []:
                row = line.rsplit()   
                for j in range(len(row)):
                    data[j].append(float(row[j]))     
    return data

In [3]:
Nx, Nt = 8, 8
block_x, block_t = 2, 2
x_elements, t_elements = int(Nx/block_x), int(Nt/block_t)
Ntest, Nagg = 5, 2*(block_x*block_t)
Ntot = Nx*Nt
Coords = np.zeros((Nx,Nt,2),dtype=int)
for x in range(Nx):
    for t in range(Nt):
        for s in range(2):
            Coords[x,t,s] = x * Nt * 2 + t * 2 + s
XCoord = np.zeros(2*Nx*Nt,dtype=int)
TCoord = np.zeros(2*Nx*Nt,dtype=int)
SCoord = np.zeros(2*Nx*Nt,dtype=int)
Agg = np.zeros((2*block_x*block_t,x_elements*t_elements),dtype=int)
print("Lattice dimensions Nx={0}, Nt={1}".format(Nx,Nt))
print("Number of aggregates {0}".format(Nagg))
print("Block_x={0}, Block_t={1}".format(block_x,block_t))
print("Number of test vectors {0}".format(Ntest))

Lattice dimensions Nx=8, Nt=8
Number of aggregates 8
Block_x=2, Block_t=2
Number of test vectors 5


In [4]:
#Computing the aggregates
for x in range(block_x):
    for t in range(block_t):
        for s in range(2):
            x0, t0 = x * x_elements, t * t_elements
            x1, t1 = (x+1) * x_elements, (t+1) * t_elements
            aggregate = x * block_t * 2 + t * 2 + s
            count = 0
            for X in range(x0,x1):
                for T in range(t0,t1):
                    i = X * Nt * 2 + T * 2 + s
                    XCoord[i] , TCoord[i], SCoord[i] = X, T, s
                    Agg[aggregate][count] = i
                    count += 1

In [5]:
class AMG():
    def __init__(self,D,nu1,nu2):
        self.test_vectors = np.zeros((Ntest,2*Ntot),dtype=complex) 
        self.P = np.zeros((2*Ntot,Ntest*Nagg),dtype=complex) 
        self.PH = np.zeros((Ntest*Nagg,2*Ntot),dtype=complex) 
        self.D = csc_array(D) #Dirac matrix
        self.Dc = np.zeros((Ntest*Nagg,Ntest*Nagg),dtype=complex) 
        self.nu1 = nu1
        self.nu2 = nu2
        self.it = 0 #Parameter to measure iterations for convergence

    def get_p(self):
        return self.P, self.PH
    def get_Dc(self):
        return self.Dc
    def get_iterations(self):
        return self.it

    def orthonormalize(self):
        v_chopped = [] #Columns of the interpolator
        for i in range(Ntest*Nagg):
            v = np.eye(1,Ntest*Nagg,i).reshape(Ntest*Nagg)
            v_chopped.append(self.P @ v)
        v_chopped = np.array(v_chopped)        
        for i in range(Nagg):
            for nt in range(Ntest):
                for j in range(nt):
                    #vdot(a,b) = a* . b /= b* . a = vdot(b,a)
                    proj = np.vdot(v_chopped[j * Nagg + i],v_chopped[nt * Nagg + i])
                    v_chopped[nt * Nagg + i] -= proj * v_chopped[j * Nagg + i]
                v_chopped[nt*Nagg+i] /= np.linalg.norm(v_chopped[nt*Nagg+i])
        matrix =  np.zeros((2*Ntot,Ntest*Nagg),dtype=complex) 
        for i in range(Ntest*Nagg):
            matrix[:,i] = v_chopped[i]
        self.P = matrix
        self.PH = csc_array(np.conjugate(np.transpose(self.P)))
        self.P = csc_array(self.P)
        self.Dc = (self.PH @(self.D @ self.P)) #matrix multiplication

    def tv(self,Nit):  
        test_vectors_copy = np.zeros((Ntest,2*Ntot),dtype=complex) 
        for i in range(Ntest): 
            for j in range(2*Ntot):
                theta = np.random.uniform(0,2*pi)
                self.test_vectors[i,j] = np.cos(theta) + 1j*np.sin(theta)
        for i in range(Ntest):
            self.test_vectors[i], exit_code = gmres(self.D, self.test_vectors[i], x0=self.test_vectors[i] ,maxiter=20)
        self.P_v() #Assemble P
        self.orthonormalize() #Orthonormalize
        print("Improving interpolator")
        for n in range(Nit):
            print("n={0} out of Nit={1}".format(n,Nit))
            for i in range(Ntest):
                self.test_vectors[i] = self.TwoGrid(1,1e-10,self.test_vectors[i],self.test_vectors[i],False)
            self.P_v() #Assemble P
            self.orthonormalize() #Orthonormalize
            
             
    def P_v(self):
        #Assembling the interpolator
        for can_vec in range(Ntest*Nagg):   
            P_v = np.zeros(2*Ntot,dtype=complex)
            v = np.eye(1,Ntest*Nagg,can_vec).reshape(Ntest*Nagg)
            for j in range(Ntest*Nagg):
                k = int(j / Nagg)
                a = int(j % Nagg)
                for i in range(len(Agg[a])):
                    x, t, s = XCoord[Agg[a,i]], TCoord[Agg[a,i]], SCoord[Agg[a,i]]
                    P_v[Coords[x,t,s]]  += self.test_vectors[k,Coords[x,t,s]] * v[j]
            self.P[:,can_vec] = P_v   
        self.PH = csc_array(np.conjugate(np.transpose(self.P)))
        self.P = csc_array(self.P)
        self.Dc = (self.PH @(self.D @ self.P)) #matrix multiplication
        
    def TwoGrid(self,max_iter,tol,x0,phi,message):
        #Solve D x = phi
        x = x0
        err = 1
        k = 0
        r0 = phi - self.D.dot(x)
        max_err = tol*np.linalg.norm(phi)
        while(k < max_iter and err > max_err):
            if self.nu1 > 0:
                x, exitCode = gmres(self.D, phi, x0=x , tol=1e-10, atol=0.0, maxiter=nu1) 
            #Coarse grid correction
            Pt_r = self.PH.dot( (phi-self.D.dot(x)))
            Dc_inv, exit_code = bicgstab(self.Dc, Pt_r,x0=Pt_r,maxiter=10000, tol=1e-10, atol=0.0) #Dc^-1 P^H(phi-Dx)
            if message==True:
                if exit_code == 0:
                    print("Bi-cgstab converged for Dc")
                else:
                    print("Bi-cgstab did NOT converge for Dc")
            x += self.P.dot(Dc_inv)
            if self.nu2>0:
                x, exitCode = gmres(self.D, phi, x0=x ,tol=1e-10, atol=0.0, maxiter=nu2)
            r = phi - self.D.dot(x)
            err = np.sqrt(np.real(np.vdot(r,r))) #complex dot product
            if message == True:
                print("iteration",k,"error",err)
            if err<=max_err:
                self.it = k+1
                print("two-grid converged in",k+1,"iterations. Error ",err)
                return x
            k+=1
        if message == True:
            print("two-grid did not converge in",max_iter,"iterations. Error ",err)
        self.it = max_iter
        return x

    def print_p(self):
        for i in range(2*Ntot):
            for j in range(Ntest*Nagg):
                print(self.P[i,j], end = " ")
            print('')

In [6]:
dim = Nx*Nt*2
print(dim)
D = np.zeros((dim,dim),dtype=complex)
row, col, re, im = DatosMat("build/D36.dat")
for i in range(dim*dim):
    D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
phi = np.zeros(dim,dtype=complex)
row, col, re, im = DatosMat("build/phi36.dat")
for i in range(dim):
    phi[i] = re[i] + 1j*im[i] 

128


In [7]:
#Initial solution
x0 = np.zeros(2*Ntot,dtype=complex)
for i in range(2*Ntot):  
    theta = np.random.uniform(0,2*pi)
    x0[i] = np.cos(theta) + 1j*np.sin(theta)

In [8]:
Dsparse = csc_array(D)
x_gmres = x0
#GMRES
tol = 1e-10
print("---GMRES---")
for i in range(1000):
    x_gmres, exit_code = gmres(Dsparse, phi, x0=x_gmres, tol=1e-10, atol=0.0, maxiter=1)
    r = phi - Dsparse.dot(x_gmres)
    err = np.sqrt(np.real(np.vdot(r,r)))
    print(i,err,exit_code)
    if err<=tol*np.linalg.norm(phi):
        print("GMRES converged in {0} iterations".format(i+1))
        break
#bi-cgstab
print("---Bi-cgstab---")
x_bi = x0
x_bi, exit_code = bicgstab(Dsparse, phi,x0=x_bi,maxiter=500, tol=1e-10)
if exit_code == 0:
    print("Converged")
else:
    print("Did not converge")
r = phi - Dsparse.dot(x_bi)
err = np.sqrt(np.real(np.vdot(r,r)))
print("Bi-cgstab error {0}".format(err))

---GMRES---
0 4.041879487629367 1
1 3.2772304610238083 1
2 2.5697510035409525 1
3 2.230988670017254 1
4 2.1050599465240514 1
5 2.024298987733361 1
6 1.9596151585141197 1
7 1.8943118862793757 1
8 1.8260411571063249 1
9 1.756889565109724 1
10 1.68584626882154 1
11 1.631279563296856 1
12 1.5817301814775855 1
13 1.5353461196466631 1
14 1.4838766757733999 1
15 1.4279573724435566 1
16 1.3688444859025997 1
17 1.329616475540076 1
18 1.3081162454042259 1
19 1.292455389321972 1
20 1.2739434287011107 1
21 1.2479169671844705 1
22 1.225694825635712 1
23 1.2086718670858874 1
24 1.1945503372102952 1
25 1.1866111814521743 1
26 1.180340133313093 1
27 1.175300412424421 1
28 1.16974636946447 1
29 1.1626744039675747 1
30 1.1528635166552497 1
31 1.1376903895012564 1
32 1.1157462411334669 1
33 1.073076312040016 1
34 1.0245093347149132 1
35 0.9845978565803516 1
36 0.9413187710410018 1
37 0.92205961624591 1
38 0.9131759028726956 1
39 0.9067545552324834 1
40 0.9008456380242484 1
41 0.8936576899692509 1
42 0.88

In [9]:
x0 = np.zeros(2*Ntot,dtype=complex)
for i in range(2*Ntot):  
    theta = np.random.uniform(0,2*pi)
    x0[i] = np.cos(theta) + 1j*np.sin(theta)
nu1, nu2 = 0, 2
Nit = 3
amg = AMG(D,nu1,nu2)
amg.tv(Nit)
P, PH = amg.get_p()
max_iter = 100
tol = 1e-10
x_sol = amg.TwoGrid(max_iter,tol,x0,phi,True)

Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
Bi-cgstab converged for Dc
iteration 0 error 0.4133956047252367
Bi-cgstab converged for Dc
iteration 1 error 0.010746078232563454
Bi-cgstab converged for Dc
iteration 2 error 0.00021769322921445097
Bi-cgstab converged for Dc
iteration 3 error 8.802770663949672e-06
Bi-cgstab converged for Dc
iteration 4 error 1.856436896201691e-07
Bi-cgstab converged for Dc
iteration 5 error 4.370563167974008e-09
Bi-cgstab converged for Dc
iteration 6 error 1.1086753940101427e-09
two-grid converged in 7 iterations. Error  1.1086753940101427e-09


C:\Users\jafan\anaconda3\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [10]:
for i in range(2*Ntot):
    line_new = '{:>12}  {:>12}'.format(x_sol[i], x_gmres[i])
    print(line_new)

(-2.1726249052054336+2.1802889463334734j)  (-2.1726249055465083+2.1802889465163147j)
(1.208795568563387+1.5457706873720471j)  (1.2087955684324825+1.5457706877394737j)
(-4.813209192441747-2.2441434290525115j)  (-4.813209193316939-2.24414342939905j)
(-0.5802144654553681+3.1143858996485485j)  (-0.5802144658207903+3.1143858996851996j)
(1.7135761073650704+1.4182942189339929j)  (1.7135761077915654+1.418294219474975j)
(0.06846565813443416-3.4215391427640647j)  (0.06846565834462937-3.421539143189889j)
(-1.2644788056150855+1.1841018864537018j)  (-1.2644788061866805+1.184101886772839j)
(1.8163703446458217-0.4407392577409726j)  (1.816370344523054-0.4407392578339342j)
(-0.28838562326478284-0.9913864426180462j)  (-0.2883856234317748-0.9913864431385768j)
(0.254697481394958-0.35986914128049746j)  (0.25469748162958294-0.35986914136134784j)
(0.24022115482063347+0.43737908934662856j)  (0.24022115518645865+0.4373790890125523j)
(-0.7241292078074019+0.21031081732642115j)  (-0.7241292076826945+0.21031081730

### Some tests for a two-grid method using GRMES as smoother

In [12]:
dim = Nx*Nt*2
D = np.zeros((dim,dim),dtype=complex)
phi = np.zeros(dim,dtype=complex)
x0 = np.zeros(2*Ntot,dtype=complex)
iterations = []
for conf in range(50):
    row, col, re, im = DatosMat("build/D{0}.dat".format(conf))
    for i in range(dim*dim):
        D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
    row, col, re, im = DatosMat("build/phi{0}.dat".format(conf))
    for i in range(dim):
        phi[i] = re[i] + 1j*im[i] 
    for i in range(2*Ntot):  
        theta = np.random.uniform(0,2*pi)
        x0[i] = np.cos(theta) + 1j*np.sin(theta)
    nu1, nu2 = 0, 2
    Nit = 3
    print("Conf #{0}".format(conf))
    amg = AMG(D,nu1,nu2)
    amg.tv(Nit)
    P, PH = amg.get_p()
    max_iter = 100
    tol = 1e-10
    x_sol = amg.TwoGrid(max_iter,tol,x0,phi,False)
    iterations.append(amg.get_iterations())
iterations = np.array(iterations)
print("Number of iterations: {0} +- {1}".format(np.round(np.mean(iterations),4), np.round(np.std(iterations)/np.sqrt(50),4  ) ))

Conf #0
Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
two-grid converged in 6 iterations. Error  9.489338785611467e-10


C:\Users\jafan\anaconda3\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Conf #1
Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
two-grid converged in 8 iterations. Error  1.112435510796934e-09
Conf #2
Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
two-grid converged in 9 iterations. Error  1.0703930183988648e-09
Conf #3
Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
two-grid converged in 12 iterations. Error  1.0689696594752356e-09
Conf #4
Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
two-grid converged in 14 iterations. Error  1.1087117495976607e-09
Conf #5
Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
two-grid converged in 19 iterations. Error  1.1278068785280695e-09
Conf #6
Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
two-grid converged in 10 iterations. Error  1.113558309660425e-09
Conf #7
Improving interpolator
n=0 out of Nit=3
n=1 out of Nit=3
n=2 out of Nit=3
two-grid converged in 13 ite

### Tests using GMRES

In [14]:
dim = Nx*Nt*2
D = np.zeros((dim,dim),dtype=complex)
phi = np.zeros(dim,dtype=complex)
x0 = np.zeros(2*Ntot,dtype=complex)
tol = 1e-10
iterations = []
for conf in range(50):
    row, col, re, im = DatosMat("build/D{0}.dat".format(conf))
    for i in range(dim*dim):
        D[ int(row[i]), int(col[i]) ] = re[i] + 1j*im[i] 
    row, col, re, im = DatosMat("build/phi{0}.dat".format(conf))
    for i in range(dim):
        phi[i] = re[i] + 1j*im[i] 
    for i in range(2*Ntot):  
        theta = np.random.uniform(0,2*pi)
        x0[i] = np.cos(theta) + 1j*np.sin(theta)
    Dsparse = csc_array(D)
    x_gmres = x0
    #GMRES
    print("Conf #{0}".format(conf))
    max_err = tol*np.linalg.norm(phi)
    err = 1
    k = 0
    while k<1000 and err>max_err:
        x_gmres, exit_code = gmres(Dsparse, phi, x0=x_gmres, tol=1e-10, atol=0.0, maxiter=1)
        r = phi - Dsparse.dot(x_gmres)
        err = np.sqrt(np.real(np.vdot(r,r)))
        if err<=max_err:
            print("GMRES converged in {0} iterations".format(k+1))
            iterations.append([k+1])
        k += 1
iterations = np.array(iterations)
print("Number of iterations: {0} +- {1}".format(np.round(np.mean(iterations),4), np.round(np.std(iterations)/np.sqrt(50),4  ) ))       

Conf #0
GMRES converged in 522 iterations
Conf #1
Conf #2
Conf #3
Conf #4
Conf #5
Conf #6
Conf #7
Conf #8
Conf #9
Conf #10
Conf #11
Conf #12
GMRES converged in 82 iterations
Conf #13
GMRES converged in 92 iterations
Conf #14
GMRES converged in 83 iterations
Conf #15
GMRES converged in 79 iterations
Conf #16
GMRES converged in 76 iterations
Conf #17
Conf #18
GMRES converged in 56 iterations
Conf #19
GMRES converged in 111 iterations
Conf #20
GMRES converged in 176 iterations
Conf #21
GMRES converged in 223 iterations
Conf #22
GMRES converged in 204 iterations
Conf #23
GMRES converged in 361 iterations
Conf #24
GMRES converged in 535 iterations
Conf #25
GMRES converged in 570 iterations
Conf #26
GMRES converged in 554 iterations
Conf #27
Conf #28
Conf #29
Conf #30
Conf #31
GMRES converged in 64 iterations
Conf #32
Conf #33
Conf #34
GMRES converged in 660 iterations
Conf #35
GMRES converged in 445 iterations
Conf #36
GMRES converged in 419 iterations
Conf #37
GMRES converged in 484 iterat